In [1]:
from qdrant_client import QdrantClient, models

d:\RAG\Qdrant_deep_dive\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

client = QdrantClient(
    url=os.getenv('QDRANT_URL'),
    api_key=os.getenv('QDRANT_API_KEY')
)

In [ ]:
# Define the collection name
collection_name = "sparse_vectors_collection"

# Create the collection with sparse vectors
client.create_collection(
    collection_name=collection_name,
    sparse_vectors_config={ #vector named "sparse_vector" all sparse vectors should be named vectors
        "sparse_vector": models.SparseVectorParams(),
    },
)

True

In [5]:
collection_name = "sparse_vectors_collection"

# Insert vectors into the collection
client.upsert(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=1,
            payload={},
            vector={ #vector named "sparse_vector"
                "sparse_vector": models.SparseVector(
                    indices=[1, 2, 3], #uint32, from 0 to 4_294_967_295
                    values=[0.2, -0.2, 0.2] #stored as floats
                )
            },
        ),
        models.PointStruct(
            id=2,
            payload={},
            vector={ #vector named "sparse_vector"
                "sparse_vector": models.SparseVector(
                    indices=[1, 5], #uint32, from 0 to 4_294_967_295
                    values=[0.1, 0.1] #stored as floats
                )
            },
        ),
    ],
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [8]:
collection_name = "sparse_vectors_collection"

client.query_points(
    collection_name=collection_name,
    using="sparse_vector",  # we need to specify the name of our sparse vectors to search against them
    limit=1,                # return the top 1 most similar result
    query=models.SparseVector(
        indices=[1, 3],
        values=[1, 1]
    ),
    with_vectors=True # to see the top 1 most similar vector
)


QueryResponse(points=[ScoredPoint(id=1, version=1, score=0.4, payload={}, vector={'sparse_vector': SparseVector(indices=[1, 2, 3], values=[0.2, -0.2, 0.2])}, shard_key=None, order_value=None)])

Let’s understand why we got Point 1 as the answer.

In the collection, we have two points:

Point 1 has three non-zero values: values = [0.2, -0.2, 0.2] with indices = [1, 2, 3]
Point 2 has two non-zero values: values = [0.1, 0.1] with indices = [1, 5]
Our query has indices = [1, 3] with corresponding values = [1, 1].

The similarity score for sparse vectors is calculated by comparing only the matching indices shared between the query and the points: [1, 3] for Point 1, and [1] for Point 2.

We multiply the corresponding values and sum them up:

score(query, Point 1) = 1 * 0.2 + 1 * 0.2 = 0.4
score(query, Point 2) = 1 * 0.1 = 0.1
Since 0.4 is higher than 0.1, Point 1 is more similar to our query.

Search on sparse vectors is always exact

# Using bm25


In [9]:
client.create_collection(
    collection_name='bm25_sparse_vectors_collection',
    sparse_vectors_config={
        "bm25_sparse_vector": models.SparseVectorParams(
            modifier=models.Modifier.IDF #Inverse Document Frequency. Once enabled, IDF is maintained at the collection level.
        ),
    },
)

True

In [9]:
grocery_items_descriptions = [
    "Grated hard cheese",
    "Fresh mozzarella cheese",
    "Whole milk yogurt",
    "Organic eggs",
    "Sourdough bread loaf",
    "Mac and cheese",
]

In [10]:

#Estimating the average length of documents in the corpus
avg_document_length = sum(len(description.split()) for description in grocery_items_descriptions) / len(grocery_items_descriptions)

client.upsert(
    collection_name='bm25_sparse_vectors_collection',
    points=[
        models.PointStruct(
            id=i,
            payload={"text": description},
            vector={
                "bm25_sparse_vector": models.Document(
                    text=description,
                    model="Qdrant/bm25",
                    options={"avg_len": avg_document_length} #To pass BM25 parameters, here we're using default k & b for the BM25 formula
                )
           },
        ) for i, description in enumerate(grocery_items_descriptions)
    ],
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [11]:
client.query_points(
    collection_name='bm25_sparse_vectors_collection',
    using="bm25_sparse_vector",
    limit=3,
    query=models.Document(
        text="cheese",
        model="Qdrant/bm25"
    ),
    with_vectors=True,
)

QueryResponse(points=[ScoredPoint(id=5, version=2, score=0.78795457, payload={'text': 'Mac and cheese'}, vector={'bm25_sparse_vector': SparseVector(indices=[1303191493, 1496964506], values=[1.1367781, 1.1367781])}, shard_key=None, order_value=None), ScoredPoint(id=0, version=2, score=0.67685914, payload={'text': 'Grated hard cheese'}, vector={'bm25_sparse_vector': SparseVector(indices=[862853134, 1277694805, 1496964506], values=[0.9765013, 0.9765013, 0.9765013])}, shard_key=None, order_value=None), ScoredPoint(id=1, version=2, score=0.67685914, payload={'text': 'Fresh mozzarella cheese'}, vector={'bm25_sparse_vector': SparseVector(indices=[274844176, 1073213896, 1496964506], values=[0.9765013, 0.9765013, 0.9765013])}, shard_key=None, order_value=None)])

## Step 7: Create a Collection for Sparse Neural Retrieval with SPLADE++

Note that we’re not configuring the Inverse Document Frequency (IDF) modifier here, unlike in BM25-based retrieval. SPLADE models don’t rely on corpus-level statistics like IDF to estimate word relevance. Instead, they generate term weights in sparse representations based on their interactions within the encoded text.

In [12]:
client.create_collection(
    collection_name="splade_vectors_collection",
    sparse_vectors_config={
        "splade_sparse_vector": models.SparseVectorParams(),
    },
)

True

In [12]:
client.upsert(
    collection_name="splade_vectors_collection",
    points=[
        models.PointStruct(
            id=i,
            payload={"text": description}, #meta data, descriptions text in human-readable format
            vector={
                "splade_sparse_vector": models.Document( #to run FastEmbed under the hood
                    text=description,
                    model="prithivida/Splade_PP_en_v1"
                )
           },
        ) for i, description in enumerate(grocery_items_descriptions)
    ],
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [13]:
client.query_points(
    collection_name="splade_vectors_collection",
    using="splade_sparse_vector",
    limit=3,
    query=models.Document(
        text="A not soft cheese",
        model="prithivida/Splade_PP_en_v1"
    ),
    with_vectors=True,
)

QueryResponse(points=[ScoredPoint(id=0, version=2, score=15.463482, payload={'text': 'Grated hard cheese'}, vector={'splade_sparse_vector': SparseVector(indices=[1010, 2081, 2524, 2828, 3067, 3528, 4383, 4435, 6211, 8808, 9841, 11825, 21774, 24665], values=[0.51333773, 0.26326102, 2.4472122, 0.30004033, 0.12786251, 0.79050905, 2.211527, 0.31894565, 1.1508049, 2.51135, 0.5997427, 0.99693406, 0.12898666, 2.4744098])}, shard_key=None, order_value=None), ScoredPoint(id=5, version=2, score=13.2577, payload={'text': 'Mac and cheese'}, vector={'splade_sparse_vector': SparseVector(indices=[1004, 1010, 1998, 2030, 2081, 2833, 3528, 4435, 4489, 4521, 4825, 6097, 6207, 8808, 9440, 9841, 11825], values=[0.7900756, 0.3015864, 1.7403926, 0.010728066, 0.33394834, 0.40943024, 0.48172736, 0.4619382, 0.11402316, 0.07198032, 0.055049647, 3.0968075, 1.1688995, 3.042247, 0.1626582, 0.94423723, 1.5706416])}, shard_key=None, order_value=None), ScoredPoint(id=1, version=2, score=9.702752, payload={'text': 'Fr

In [14]:
client.query_points(
    collection_name="bm25_sparse_vectors_collection",
    using="bm25_sparse_vector",
    limit=3,
    query=models.Document(
        text="A not soft cheese",
        model="Qdrant/bm25"
    ),
    with_vectors=True,
)

QueryResponse(points=[ScoredPoint(id=5, version=2, score=0.78795457, payload={'text': 'Mac and cheese'}, vector={'bm25_sparse_vector': SparseVector(indices=[1303191493, 1496964506], values=[1.1367781, 1.1367781])}, shard_key=None, order_value=None), ScoredPoint(id=0, version=2, score=0.67685914, payload={'text': 'Grated hard cheese'}, vector={'bm25_sparse_vector': SparseVector(indices=[862853134, 1277694805, 1496964506], values=[0.9765013, 0.9765013, 0.9765013])}, shard_key=None, order_value=None), ScoredPoint(id=1, version=2, score=0.67685914, payload={'text': 'Fresh mozzarella cheese'}, vector={'bm25_sparse_vector': SparseVector(indices=[274844176, 1073213896, 1496964506], values=[0.9765013, 0.9765013, 0.9765013])}, shard_key=None, order_value=None)])